In [3]:
import autogen
from autogen import config_list_from_json

In [4]:
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")

## Construct Agents

In [5]:
llm_config = {
    'config_list': config_list,
    'timeout': 120,
    'temperature': 0,
    # 'cache_seed': 42
}

In [6]:
user_proxy = autogen.UserProxyAgent(
    name='CEO',
    system_message="CEO. Interact with Product Manager to discuss product requirements." \
                    "Plan execution needs to be approved by CEO.",
    code_execution_config=False,
)

product_manager = autogen.AssistantAgent(
    name="Product Manager",
    system_message='''
        Product Manager. 
        Listens to suggestions from CEO and come up with a Product Requirement Design (PRD).
        The PRD consists of User Stories for the features as well as the acceptance criteria.
        Revise the PRD based on feedback from CEO, UI/UX Designer and Engineer, until CEO approval.
        The PRD may involve an Engineer who can write code and a UI/UX Designer who doesn't write code.
        Explain the PRD first. 
        Be clear which step is performed by an Engineer, and which step is performed by a UI/UX Designer.
        After the PRD is approved, share it with the Data Analyst to come up with useful product metrics and how to measure success.
    ''',
    llm_config=llm_config,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message='''
        Engineer. 
        You follow an approved plan. 
        You write python/shell code to solve tasks. 
        Wrap the code in a code block that specifies the script type. 
        The user can't modify your code. 
        So do not suggest incomplete code which requires others to modify. 
        Don't use a code block if it's not intended to be executed by the executor.
        Don't include multiple code blocks in one response. 
        Do not ask others to copy and paste the result. 
        Check the execution result returned by the executor.
        If the result indicates there is an error, fix the error and output the code again. 
        Suggest the full code instead of partial code or code changes. 
        If the error can't be fixed or if the task is not solved even after the code is executed successfully, 
        analyze the problem, revisit your assumption, collect additional info you need, 
        and think of a different approach to try.
''',
)

ui_ux_designer = autogen.AssistantAgent(
    name="UI/UX Designer",
    llm_config=llm_config,
    system_message='''
        UI/UX Designer. 
        You have good understanding of product requirements. 
        You emphatize with the users and are able to suggest good useful features.
        You are able to conduct research on product features and communicate it back to the Product Manager.
        You also criticize on features that may not add value or does not provide good user experience.
    ''',
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "code"},
)

data_analyst = autogen.AssistantAgent(
    name="Data Analyst",
    llm_config=llm_config,
    system_message='''
        Data Analyst.
        You are a critical thinker and know what metrics to look for to measure a product feature success.
        You will write a report on the possible metrics, and how they can add value to the company in terms of revenue, user engagements etc.
    ''',
)

groupchat = autogen.GroupChat(agents=[user_proxy, engineer, product_manager, data_analyst, executor, ui_ux_designer], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [7]:
user_proxy.initiate_chat(
    manager,
    message="""
come up with a job portal to search for white collar jobs targetting the Thailand market. Generate the product requirements, then prepare a pitch deck presentation too.
""",
)

CEO (to chat_manager):


come up with a job portal to search for white collar jobs targetting the Thailand market. Generate the product requirements, then prepare a pitch deck presentation too.


--------------------------------------------------------------------------------


Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  list down the possible features


CEO (to chat_manager):

list down the possible features

--------------------------------------------------------------------------------
Product Manager (to chat_manager):

**Product Requirements Document**

**Product Name:** Job Portal for White Collar Jobs in Thailand

**Version:** 1.0

**Date:** 2023-03-08

**1. Objective**

The objective of this product is to provide a platform for job seekers and employers to connect with each other. The platform will allow job seekers to search for jobs, post their resumes, and connect with employers. Employers will be able to post jobs, search for candidates, and communicate with job seekers.

**2. Target Audience**

The target audience for this product is white-collar workers in Thailand. This includes professionals with a bachelor's degree or higher who are looking for jobs in a variety of industries.

**3. Customer Needs**

Customers need a platform that allows them to:

* Search for jobs by location, industry, and keyword
* Post their resum

Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  sounds good, ask the engineer for implementation details


CEO (to chat_manager):

sounds good, ask the engineer for implementation details

--------------------------------------------------------------------------------
Engineer (to chat_manager):

**Engineering Implementation**

The job portal can be implemented using a variety of technologies. The following is a suggested architecture:

* **Frontend:** The frontend of the job portal can be built using a variety of frameworks, such as React, Angular, or Vue.js. The frontend will be responsible for rendering the user interface and handling user interactions.
* **Backend:** The backend of the job portal can be built using a variety of languages and frameworks, such as Python, Ruby, or Java. The backend will be responsible for storing and retrieving data, processing user requests, and sending email notifications.
* **Database:** The job portal will need a database to store data such as job listings, resumes, and user profiles. The database can be either relational or NoSQL.
* **Cloud:** The jo

KeyboardInterrupt: 